# CNN (Convolutional Neural Network)(Evrişimsel Sinir Ağı) veya convnet
* Evrişim bir convnet'in katmanlarına benzersiz yapılarını veren matematiksel işlemdir.
    
### The Convolutional Classifier (Evrişimsel Sınıflandırcı)
* Görüntü Sınıflandırması için kullanılan bir convnet 2 bölümden oluşur:
 * a convolutional base(evrişimsel taban)
 * a dense head(yoğun baş)
 
Taban bir görüntüden özellik çıkarmak için kullanılır.<br>
Head(baş) resmin sınıfını belirlemek için kullanılır.

### Training the Classifier (Sınıflandırıcıyı Eğitmek)
Eğitim sırasında ağın amacı 2 şeyi öğrenmektir:
1. Bir görüntüden hangi özelliklerin çıkarılacğı (base)
2. Hangi sınıf hangi özelliklere gider (head)

Önceden eğitilmiş base'e eğitimsiz bir head takıyoruz. Yani ağın yapmayı öğrenmiş olan kısmını yeniden kullanıyoruz. (Transfer Learning)

1. Özellikleri ayıkla ve öğrenmek için yeni katmanlar ekle
2. Sınıflandır

# Train a Convnet Classifier

Steps: 
1. Load data (ImageNet veri setinin applications modulündeki VGG16 modelini kullanacağız)
2. Define pretrained base
3. Attach head
4. Train

In [1]:
# 1. Load Data
# Imports
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed(31415)

# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") # to clean up output cells


# Load training and validation sets
ds_train_ = image_dataset_from_directory(
    '../input/car-or-truck/train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    '../input/car-or-truck/valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

NotFoundError: Could not find directory ../input/car-or-truck/train

In [26]:
import matplotlib.pyplot as plt

In [5]:
# 2. Define pretrained base
import tensorflow as tf
pretrained_base = tf.keras.models.load_model(
    '../input/cv-course-models/cv-course-models/vgg16-pretrained-base',
)
pretrained_base.trainable = False

OSError: No file or directory found at ../input/cv-course-models/cv-course-models/vgg16-pretrained-base

In [ ]:
# 3. Attach Head
# Dense(Yoğun) Layer (Olasılık puanına dönüştüren çıktı katmanı)
# Flatten(Düz) Layer (Head'in ihtiyaç duyduğu tek boyutlu girdileri 2 boyutlu çıktılara dönüştürür)

from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
# 4. Train (crossentropy ve accuracy ikili versiyonu kullanacağız. adam optimizer genellikle iyi performans gösterir.)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model.fit(
    ds_train,
    validaton_data = ds_valid,
    epochs=30,
    verbose=0,
)

In [3]:
# Sinir ağı eğitirken loss ve metric grafikleri incelemek iyidir.
# history nesnesi bunları içerir(history.history dicxtionary'si içinde). Pandas ile dataframe'e çevireceğiz.

import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()